<a href="https://colab.research.google.com/github/Abdulxmannan/Machine-learning/blob/main/sentence_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch

In [19]:
sentence1="#Bitcoin is up 34% since Bank of England Governor said 'Be prepared to lose all your money in BTC and crypto."
sentence2="One of the biggest bull traps I've ever seen"

In [20]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off

In [21]:
from sentence_transformers import SentenceTransformer, util

In [22]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [23]:
model.max_seq_length

384

In [24]:
corpus = [
    "#Bitcoin is up 34% since Bank of England Governor said ‘Be prepared to lose all your money in BTC and crypto.’",
    "One of the biggest bull traps I’ve ever seen",
    "How I sleep knowing ethereum is going to 10k in 2023",
    "Bitcoin is a scam",
    "IS THE $BTC BOTTOM IN? Are you team BULL or team BEAR? Do you have the DATA to prove your point??",
    "I will stop bragging about calling the top when I start bragging about calling the bottom. #bitcoin",
    "First powerlifting meet of the year and a new squat PR!!",
    "On January 9th, 2023, the American Academy of Pediatrics published new guidelines treating obesity in children and adolescents",
    "What’s worse, someone dropping the bar from the top of a deadlift or not putting their shopping cart away?",
    "The sport of powerlifting includes squat bench and deadlift. But the concept of powerlifting is Force= Mass x Acceleration."
]

In [25]:
corpus_embeddings = model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
corpus_embeddings.shape

torch.Size([10, 768])

In [42]:
query = "how much should i deadlift at 82 kg body weight"

In [43]:
query_embedding = model.encode(query, convert_to_numpy=True)

In [44]:
import torch

# Ensure that both tensors are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert NumPy arrays to PyTorch tensors and move to the same device
query_embedding = torch.tensor(query_embedding).to(device)
corpus_embedding = torch.tensor(corpus_embeddings[9]).to(device)

# Perform cosine similarity
similarity = util.cos_sim(query_embedding, corpus_embedding)


<ipython-input-44-d952fa8e8f83>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  corpus_embedding = torch.tensor(corpus_embeddings[9]).to(device)


In [45]:
util.cos_sim(query_embedding, corpus_embeddings[9])

tensor([[0.3183]], device='cuda:0')

In [46]:
sentence1_embedding = model.encode(sentence1, convert_to_tensor=True)
sentence2_embedding = model.encode(sentence2, convert_to_tensor=True)

In [47]:
util.cos_sim(sentence1_embedding, sentence2_embedding)

tensor([[0.1403]], device='cuda:0')

In [48]:
result = util.semantic_search(query_embedding, corpus_embeddings)[0]

In [49]:
result

[{'corpus_id': 9, 'score': 0.318325936794281},
 {'corpus_id': 8, 'score': 0.23216144740581512},
 {'corpus_id': 6, 'score': 0.12675632536411285},
 {'corpus_id': 7, 'score': 0.09198696911334991},
 {'corpus_id': 2, 'score': 0.03834538161754608},
 {'corpus_id': 0, 'score': 0.020931709557771683},
 {'corpus_id': 5, 'score': 0.015872733667492867},
 {'corpus_id': 4, 'score': 0.0032979692332446575},
 {'corpus_id': 3, 'score': -0.02854291908442974},
 {'corpus_id': 1, 'score': -0.052554693073034286}]

In [50]:
for item in result:
  print(round(item['score'],2), corpus[item['corpus_id']])

0.32 The sport of powerlifting includes squat bench and deadlift. But the concept of powerlifting is Force= Mass x Acceleration.
0.23 What’s worse, someone dropping the bar from the top of a deadlift or not putting their shopping cart away?
0.13 First powerlifting meet of the year and a new squat PR!!
0.09 On January 9th, 2023, the American Academy of Pediatrics published new guidelines treating obesity in children and adolescents
0.04 How I sleep knowing ethereum is going to 10k in 2023
0.02 #Bitcoin is up 34% since Bank of England Governor said ‘Be prepared to lose all your money in BTC and crypto.’
0.02 I will stop bragging about calling the top when I start bragging about calling the bottom. #bitcoin
0.0 IS THE $BTC BOTTOM IN? Are you team BULL or team BEAR? Do you have the DATA to prove your point??
-0.03 Bitcoin is a scam
-0.05 One of the biggest bull traps I’ve ever seen


In [55]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('bert-base-nli-mean-tokens')

examples = [
    InputExample(
        texts=[
            "#Bitcoin is up 34% since Bank of England Governor said 'Be prepared to lose all your money in BTC and crypto.",
            "One of the biggest bull traps I've ever seen"
        ], label= 0.9
    )

]

dataset = SentencesDataset(examples, model)


In [56]:
dataloader = DataLoader(dataset, shuffle=True)
loss = losses.CosineSimilarityLoss(model=model)

In [57]:
save_path = "trained_model"

In [58]:
model.fit(train_objectives=[(dataloader, loss)],epochs=10, warmup_steps=10, output_path=save_path)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
trained_model = SentenceTransformer(save_path)

In [60]:
sentence1_embedding = trained_model.encode(sentence1, convert_to_tensor=True)
sentence2_embedding = trained_model.encode(sentence2, convert_to_tensor=True)

In [61]:
util.cos_sim(sentence1_embedding, sentence2_embedding)

tensor([[0.9368]], device='cuda:0')